In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
pip install datatable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 6.8 MB/s eta 0:00:00


In [6]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("./drive/MyDrive/millennium_project")

import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit
from utils import *

from IPython.display import clear_output

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# TF
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [7]:
ticker_name = "GOOG"

In [8]:
N_FOLD = 5
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = ticker_name+'-wavenet'
DATA_NAME = ticker_name
mkdir(f'./drive/MyDrive/millennium_project/models/{SOL_NAME}/')

# Functions

In [9]:
def rmse_keras(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))
def cbr(x, out_layer, kernel, stride, dilation):
        x = tf.keras.layers.Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)
        return x

def WaveNetResidualConv1D(num_filters, kernel_size, stacked_layer):

    def build_residual_block(l_input):
        resid_input = l_input
        skip_connections = []
        for dilation_rate in [2**i for i in range(stacked_layer)]:
            l_sigmoid_conv1d = tf.keras.layers.Conv1D(
              num_filters, kernel_size, dilation_rate=dilation_rate,
              padding='same', activation='sigmoid')(l_input)
              #padding='same', activation='linear')(l_input)
            l_tanh_conv1d = tf.keras.layers.Conv1D(
             num_filters, kernel_size, dilation_rate=dilation_rate,
             padding='same', activation='tanh')(l_input)
             #padding='same', activation='mish')(l_input)
            l_input = tf.keras.layers.Multiply()([l_sigmoid_conv1d, l_tanh_conv1d])
            l_skip = tf.keras.layers.Conv1D(num_filters, 1, padding='same')(l_input)
            skip_connections.append(l_skip)
            l_input = tf.keras.layers.Conv1D(num_filters, 1, padding='same')(l_input)
            resid_input = tf.keras.layers.Add()([resid_input ,l_input])
        return resid_input ,skip_connections
    return build_residual_block

def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = tf.keras.layers.Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = tf.keras.layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'tanh',
                              dilation_rate = dilation_rate)(x)
            sigm_out = tf.keras.layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid',
                              dilation_rate = dilation_rate)(x)
            x = tf.keras.layers.Multiply()([tanh_out, sigm_out])
            x = tf.keras.layers.Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = tf.keras.layers.Add()([res_x, x])
        return res_x

def create_wavenet(num_columns, num_labels, learning_rate, filters, kernel_size, n):
    num_filters_ = filters[0]

    # input
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)

    # 1dcnn
    x = tf.keras.layers.Dense(4096, activation='relu')(x)
    x = tf.keras.layers.Reshape((64, 64))(x)

    x1 = tf.keras.layers.Dense(256, activation='linear', name='in_dense')(x)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x2 = tf.keras.layers.Dense(256, activation='linear', name='in_dense2')(x)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x4 = tf.keras.layers.Conv1D(32, 3,activation='linear', padding='same')(x)
    x4 = tf.keras.layers.Conv1D(64, 3,activation='linear', padding='same')(x4)


    x5 = tf.keras.layers.Conv1D(32, 7,activation='linear', padding='same')(x)
    x5 = tf.keras.layers.Conv1D(64, 7,activation='linear', padding='same')(x5)

    x6 = tf.keras.layers.Conv1D(16, 3,activation='tanh', padding='same')(x)
    x6 = tf.keras.layers.Conv1D(32, 3,activation='tanh', padding='same')(x6)
    x6 = tf.keras.layers.Conv1D(64, 3,activation='tanh', padding='same')(x6)

    x7 = tf.keras.layers.Conv1D(16, 7,activation='tanh', padding='same')(x)
    x7 = tf.keras.layers.Conv1D(32, 7,activation='tanh', padding='same')(x7)
    x7 = tf.keras.layers.Conv1D(64, 7,activation='tanh', padding='same')(x7)

    x_merge = tf.keras.layers.Concatenate()([x4, x5,x6,x7])
    x = tf.keras.layers.Add()([x1,x2,x_merge])


    x00 = tf.keras.layers.Conv1D(num_filters_, 1, padding='same')(x)
    x,skip1 = WaveNetResidualConv1D(num_filters_, kernel_size[0], int(n[0]/2))(x00)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv1D(num_filters_, 1, padding='same')(x)

    x,skip2 = WaveNetResidualConv1D(num_filters_, kernel_size[1], int(n[1]))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv1D(num_filters_, 1, padding='same')(x)

    x,skip3 = WaveNetResidualConv1D(num_filters_, kernel_size[2], int(n[2]))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv1D(num_filters_, 1, padding='same')(x)

    x,skip4 = WaveNetResidualConv1D(num_filters_, kernel_size[3], 1)(x)

    skip_cons = skip4+skip1+skip2+skip3
    x = tf.keras.layers.Add()(skip_cons)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(num_labels)(x)

    model = tf.keras.models.Model(inputs=inp, outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=rmse_keras,
    )
    return model


# Loading data

In [10]:
# train
df_train = dt.fread(f'./drive/MyDrive/millennium_project/dataset/train_{DATA_NAME}_LGB.csv').to_pandas()
fea_cols = fea_cols_GOOG
df_train["name"] = ticker_name
# result
df_result = dt.fread('./drive/MyDrive/millennium_project/dataset/train.csv').to_pandas()
df_result = gen_row_id(df_result)

In [11]:
df_train = gen_row_id(df_train)
df_train = add_time_fold(df_train, N_FOLD)

In [12]:
df_train.index = df_train.row_id
df_result.index = df_result.row_id

# Evaluation

In [13]:
batch_size = 1024
filters = [8]
kernel_size = [5, 2, 1, 1]
n_params = [2, 2, 4]
learning_rate = 2.4e-05
epochs = 1000

list_seeds = [0, 11, 42]

In [ ]:
list_rmse = []
for i_seed, seed in enumerate(list_seeds):
    df_train = add_time_fold(df_train, N_FOLD, seed=seed)
    list_rmse += [[]]
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]


        X_train = df_tr[fea_cols].values
        y_train = df_tr[['value_to_predict']].values
        X_test = df_te[fea_cols].values
        y_test = df_te[['value_to_predict']].values
        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_seed+1}/{len(list_seeds)} | {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)


        # Callbacks
        ckp_path = f'./models/{SOL_NAME}/model_{i_seed}_{i_fold}.hdf5'
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, min_delta=1e-5, verbose=2)
        es = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=30, restore_best_weights=True, verbose=2)

        model = create_wavenet(X_train.shape[1], 1, learning_rate, filters, kernel_size, n_params)
        history = model.fit(X_train, y_train,
            epochs=epochs,
            validation_data=(X_test, y_test),
            validation_batch_size=len(y_test),
            batch_size=batch_size,
            verbose=2,
            callbacks=[rlr, es]
        )
        # model = tf.keras.models.load_model(ckp_path, custom_objects={'mspe_loss': mspe_loss})

        y_pred = model.predict(X_test, batch_size=len(y_test))
        curr_rmse = rmse_keras(y_test, y_pred)
        list_rmse[-1] += [curr_rmse]
        model.save(ckp_path)
        # generate and save preds
        df_result.loc[idx_test, f'pred_{i_seed}'] = y_pred
        clear_output()
        print(list_rmse)

[[<tf.Tensor: shape=(), dtype=float64, numpy=0.5625915280483882>, <tf.Tensor: shape=(), dtype=float64, numpy=0.046282511612961014>, <tf.Tensor: shape=(), dtype=float64, numpy=0.04226398228644507>, <tf.Tensor: shape=(), dtype=float64, numpy=0.518031733039324>, <tf.Tensor: shape=(), dtype=float64, numpy=0.043958510491679495>], [<tf.Tensor: shape=(), dtype=float64, numpy=0.05355743765502558>]]
Fold 2/3 | 2/5 (3730, 15) (933, 15)
Epoch 1/1000
4/4 - 29s - loss: 0.8432 - val_loss: 0.7681 - lr: 2.4000e-05 - 29s/epoch - 7s/step
Epoch 2/1000
4/4 - 12s - loss: 0.5646 - val_loss: 0.7281 - lr: 2.4000e-05 - 12s/epoch - 3s/step
Epoch 3/1000
4/4 - 13s - loss: 0.4533 - val_loss: 0.7036 - lr: 2.4000e-05 - 13s/epoch - 3s/step
Epoch 4/1000
4/4 - 11s - loss: 0.4229 - val_loss: 0.6944 - lr: 2.4000e-05 - 11s/epoch - 3s/step
Epoch 5/1000
4/4 - 12s - loss: 0.3759 - val_loss: 0.6939 - lr: 2.4000e-05 - 12s/epoch - 3s/step
Epoch 6/1000
4/4 - 13s - loss: 0.3401 - val_loss: 0.6917 - lr: 2.4000e-05 - 13s/epoch - 3s

In [ ]:
df_result.to_csv(f'./drive/MyDrive/millennium_project/results/{SOL_NAME}.csv', index=False)

In [ ]:
for i in range(len(list_seeds)):
    print(i, rmse(df_result['value_to_predict'], df_result[f'pred_{i}']))
print('All: ', rmse(df_result['value_to_predict'], df_result[[f'pred_{i}' for i in range(len(list_seeds))]].mean(axis=1)))